# Pandas 进阶

## 1. Group操作：数据聚合

**1.1 生成DataFrame**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; 

In [ ]:
period = pd.date_range('2017-1-1', periods=10000, freq='D')
df = pd.DataFrame(np.random.randn(10000, 4),
                  columns=['Data1', 'Data2', 'Data3', 'Data4'],
                  index = period)

In [ ]:
df.head()      #快速预览前5行；

#### 1.2 新增一例用于Group操作

In [ ]:
df['group1'] = np.random.choice(['A', 'B', 'C', 'D'], 10000)       #np.random.choice

In [ ]:
df.tail()

In [ ]:
df.head()

**1.3 生成DataFrameGroupBy对象** 

In [ ]:
grouped = df.groupby('group1')     #df.groupby();必背；

In [ ]:
grouped

In [ ]:
type(grouped)

In [ ]:
grouped.size()           #每组元素数量

#### 1.4 Group对象的聚合运算

In [ ]:
grouped.sum()

In [ ]:
grouped.max()

In [ ]:
grouped.mean()

In [ ]:
grouped.describe()        #整体数据特征概览；

In [ ]:
np.transpose(grouped.describe() )

#### 1.5 选择Group对象中的数据

In [ ]:
grouped.get_group('A').head()    #get_group()

In [ ]:
grouped.get_group('B').head() 

In [ ]:
grouped.            #tab键查看对应其对应的方法

#### 1.6 双重Group

In [ ]:
df['group2'] = np.random.choice(['Python','C++','Java'], 10000)   

In [ ]:
df.head()

In [ ]:
grouped = df.groupby(['group1', 'group2'])
grouped.size()                                    #多组分类

In [ ]:
grouped.max()

In [ ]:
grouped.min()

In [ ]:
grouped.agg([np.max, np.sum])               #自定义聚合.分类汇总、查看，可以自定义聚合；

In [ ]:
grouped.agg({'Data1':np.mean, 'Data2':np.max})

### 2. Pandas中Concat, Join, Merge操作

#### 2.1 生成DataFrame

In [20]:
df1 = pd.DataFrame(['10', '20', '25', '30'], 
                    index=['a', 'b', 'c', 'd'],
                    columns=['PE',])
df1

PE
a  10
b  20
c  25
d  30

In [25]:
df2 = pd.DataFrame(['2.5', '3.2', '2.5'], 
                    index=['a', 'd', 'g'],
                    columns=['PB',])
df2

PB
a  2.5
d  3.2
g  2.5

#### 2.2 Concat操作

In [26]:
pd.concat((df1, df2))    #默认是直接拼接在后面；

PB   PE
a  NaN   10
b  NaN   20
c  NaN   25
d  NaN   30
a  2.5  NaN
d  3.2  NaN
g  2.5  NaN

In [27]:
pd.concat((df1, df2),axis =1)          #横向拼接concat很有用，会自动匹配索引；join(Outer),并集；

PE   PB
a   10  2.5
b   20  NaN
c   25  NaN
d   30  3.2
g  NaN  2.5

In [28]:
pd.concat((df1, df2), ignore_index=True)

PB   PE
0  NaN   10
1  NaN   20
2  NaN   25
3  NaN   30
4  2.5  NaN
5  3.2  NaN
6  2.5  NaN

#### 2.3 Join操作

是根据两张表格的索引去进行合并

In [31]:
print(df1)
print(df2)

   PE
a  10
b  20
c  25
d  30
    PB
a  2.5
d  3.2
g  2.5


In [29]:
df1.join(df2)                     # left join = default,默认按照左边表格对齐；

PE   PB
a  10  2.5
b  20  NaN
c  25  NaN
d  30  3.2

In [32]:
df1.join(df2, how='left')  # default，how控制的是按索引合并的方法；

PE   PB
a  10  2.5
b  20  NaN
c  25  NaN
d  30  3.2

In [33]:
df2.join(df1)

PB   PE
a  2.5   10
d  3.2   30
g  2.5  NaN

In [37]:
df = pd.DataFrame({'PE': df1['PE'], 'PB': df2['PB']})                    #使用DataFrame进行拼接操作
df

PB   PE
a  2.5   10
b  NaN   20
c  NaN   25
d  3.2   30
g  2.5  NaN

In [34]:
df1.join(df2, how='right')

PE   PB
a   10  2.5
d   30  3.2
g  NaN  2.5

In [35]:
df1.join(df2, how='inner')    #取交集，双方都有的才进行显示；

PE   PB
a  10  2.5
d  30  3.2

In [36]:
df1.join(df2, how='outer')    #取并集，保留双方所有的内容；

PE   PB
a   10  2.5
b   20  NaN
c   25  NaN
d   30  3.2
g  NaN  2.5

#### 2.4 Merge操作: 同等数量合并

按列来进行合并（也支持按index来进行合并）

In [39]:
df1

PE
a  10
b  20
c  25
d  30

In [38]:
df2 = pd.DataFrame(['2.5', '3.2', '2.5', '2'], 
                    index=['a', 'b', 'c','d'],
                    columns=['PB',])
df2

PB
a  2.5
b  3.2
c  2.5
d    2

In [52]:
roe = pd.Series([0.12, 0.06, 0.08,0.02], index=['a', 'b', 'c','d'])         #添加一列一列以进行合并
df1['ROE'] = roe    #新增一列用法；
df2['ROE'] = roe

In [53]:
df1

PE   ROE
a  10  0.12
b  20  0.06
c  25  0.08
d  30  0.02

In [54]:
df2

PB   ROE
a  2.5  0.12
b  3.2  0.06
c  2.5  0.08
d    2  0.02

In [55]:
pd.merge(df1, df2, on = 'ROE')        #会自动根据重复列进行merge，但是索引会重置；

PE   ROE   PB
0  10  0.12  2.5
1  20  0.06  3.2
2  25  0.08  2.5
3  30  0.02    2

In [ ]:
# Merge合并的时候如果ROE不同的话，只会合并ROE相同的数据行；
roe1 = pd.Series([0.12, 0.06, 0.08,0.02], index=['a', 'b', 'c','d']) 
roe2 = pd.Series([0.2, 0.06, 0.08,0.02], index=['a', 'b', 'c','d'])     #添加一列一列以进行合并
df1['ROE'] = roe1    #新增一列用法；
df2['ROE'] = roe2

In [58]:
df1.join(df2, lsuffix = '_l')   

PE  ROE_l   PB   ROE
a  10   0.12  2.5  0.12
b  20   0.06  3.2  0.06
c  25   0.08  2.5  0.08
d  30   0.02    2  0.02

In [64]:
pd.merge(df1, df2, left_index=True, right_index=True, how='outer',suffixes=['_df1','_df2'])    
#merge也可以通过left_index，right_index进行按index索引,可以实现等价于join的形式；

PE  ROE_df1   PB  ROE_df2
a  10     0.12  2.5     0.12
b  20     0.06  3.2     0.06
c  25     0.08  2.5     0.08
d  30     0.02    2     0.02

In [65]:
pd.merge(df1, df2, how='outer')       #ROE这一列数据两张表格相同，所以无法看出outer和其他合并类型的区别；

PE   ROE   PB
0  10  0.12  2.5
1  20  0.06  3.2
2  25  0.08  2.5
3  30  0.02    2

#### 2.5 Merge操作: 非同等数量合并

In [3]:
df2 = pd.DataFrame(['2.5', '3.2', '2.5'], 
                    index=['a', 'b', 'c'],
                    columns=['PB',])
df2

PB
a  2.5
b  3.2
c  2.5

In [2]:
df1 = pd.DataFrame(['10', '20', '25', '30'], 
                    index=['a', 'b', 'c', 'd'],
                    columns=['PE',])
df1

PE
a  10
b  20
c  25
d  30

In [4]:
roe = pd.Series([0.12, 0.08, 0.06], index=['b', 'd', 'c'])         #添加一列一列以进行合并
df1['ROE'] = roe
df2['ROE'] = roe

In [5]:
df1

PE   ROE
a  10   NaN
b  20  0.12
c  25  0.06
d  30  0.08

In [6]:
df2

PB   ROE
a  2.5   NaN
b  3.2  0.12
c  2.5  0.06

In [12]:
pd.merge(df1, df2)                #join是按index合并，merge是按列合并；默认是inner

PE   ROE   PB
0  10   NaN  2.5
1  20  0.12  3.2
2  25  0.06  2.5

In [13]:
pd.merge(df1, df2, on='ROE')  # default

PE   ROE   PB
0  10   NaN  2.5
1  20  0.12  3.2
2  25  0.06  2.5

In [14]:
pd.merge(df1, df2, how='outer')

PE   ROE   PB
0  10   NaN  2.5
1  20  0.12  3.2
2  25  0.06  2.5
3  30  0.08  NaN

In [18]:
df1.join(df2,rsuffix = '_df2', how = 'left')   

PE   ROE   PB  ROE_df2
a  10   NaN  2.5      NaN
b  20  0.12  3.2     0.12
c  25  0.06  2.5     0.06
d  30  0.08  NaN      NaN

In [20]:
pd.merge(df1, df2, left_index=True, right_index=True, how='left')    
#merge也可以通过left_index，right_index进行按index索引

PE  ROE_x   PB  ROE_y
a  10    NaN  2.5    NaN
b  20   0.12  3.2   0.12
c  25   0.06  2.5   0.06
d  30   0.08  NaN    NaN

In [21]:
df1 = pd.DataFrame(['10', '20', '25', '30'], 
                    index=['a', 'b', 'c', 'd'],
                    columns=['PE',])
df1

PE
a  10
b  20
c  25
d  30

In [22]:
df2 = pd.DataFrame(['2.5', '3.2', '2.5'], 
                    index=['a', 'b', 'c'],
                    columns=['PB',])
df2

PB
a  2.5
b  3.2
c  2.5

In [23]:
roe = pd.Series([0.12, 0.08, 0.06], index=['b', 'd', 'c'])         #添加一列一列以进行合并
df1['ROE1'] = roe
df2['ROE2'] = roe

In [24]:
df1

PE  ROE1
a  10   NaN
b  20  0.12
c  25  0.06
d  30  0.08

In [25]:
df2

PB  ROE2
a  2.5   NaN
b  3.2  0.12
c  2.5  0.06

In [27]:
pd.merge(df1, df2, left_on='ROE1', right_on='ROE2')     #两张DataFrame列名是不同的，但是表示是同一个的意思；

PE  ROE1   PB  ROE2
0  10   NaN  2.5   NaN
1  20  0.12  3.2  0.12
2  25  0.06  2.5  0.06

### 3. 层次化索引

In [28]:
df = pd.Series(np.random.randn(5),                 #定义层次化索引
               index=[['a', 'a', 'b','b', 'b'],
                      [1,2,1,2,3]])

In [29]:
df

a  1   -0.541225
   2    1.499903
b  1   -0.578726
   2    0.161935
   3    0.097446
dtype: float64

In [30]:
df.index

MultiIndex(levels=[['a', 'b'], [1, 2, 3]],
           labels=[[0, 0, 1, 1, 1], [0, 1, 0, 1, 2]])

In [31]:
df['b']                       # 使用层次化索引

1   -0.578726
2    0.161935
3    0.097446
dtype: float64

In [32]:
df['a']   

1   -0.541225
2    1.499903
dtype: float64

层次化索引在数据重塑中的应用

In [33]:
df.unstack()                 # 使用unstack方法将数据重新排列

1         2         3
a -0.541225  1.499903       NaN
b -0.578726  0.161935  0.097446

In [34]:
df.unstack().stack()         # stack是unstack方法的逆运算

a  1   -0.541225
   2    1.499903
b  1   -0.578726
   2    0.161935
   3    0.097446
dtype: float64

使用层次化索引进行聚合运算

In [35]:
df.sum(level=0)             # 按照level=0进行聚合运算

a    0.958678
b   -0.319345
dtype: float64

In [37]:
df.groupby(level=0).sum()           # 按照level=0进行聚合运算

a    0.958678
b   -0.319345
dtype: float64

In [36]:
df.sum(level=1) 

1   -1.119952
2    1.661838
3    0.097446
dtype: float64

声明：本资料仅限内部研究使用，切勿外传。